In [2]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import numpy as np

In [12]:
ctx = snowflake.connector.connect(user='juan.yam@ext.cemex.com',
                                     account='hg45590', 
                                     region = 'East-US-2.azure',
                                     warehouse='PRD_LND_MRP_SAP', 
                                     database='PRD_TDS_MX_PLANOPER_CONCRETO',       
                                     schema ='EXOP',
                                     authenticator="externalbrowser",
                                     autocommit=True)   

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [19]:
cs = ctx.cursor()
cs.execute("DROP TABLE IK17" )
cs.close()

In [ ]:
cs.execute('CREATE TABLE "IK172" ("Linea_seleccionada" text, "Documento_medicion" text,	"Punto_de_medida" text, "Fecha" text, "Hora_medicion" text,	 "ValMed_Val_total_cont" text, "UnidadCaracteristica" text,	 "Creado_por" text,	 "Posicion_de_medida" text,	"Diferencia_val_cont" text,	"Equipo" text,	"Texto1" text,"Texto2" text)')


# Procesamiento de los datos de las bases

In [266]:
# FUNCIONES
def Columnames(database, namebase):
    deletecol = 0
    a = 0
    a = database.columns
    a = list(a)
    deletecol = [lista for lista in a if "Unnamed" in lista]
    if(deletecol != 0):
        database = database.drop(columns = deletecol)
    
    database = database.loc[0:9,:]
    database= database.dropna(how='all')
    new_names = []
    for i in database.columns.values:
        result = i.replace(' ','_')
        result = result + namebase
        new_names.append(result)
    database.columns = new_names
    return database

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [74]:
df = pd.read_excel("DELIV.xlsx", sheet_name='Sheet1')
df2 = pd.read_excel("DSERV.xlsx", sheet_name='Sheet1')
df3 = pd.read_excel("EMP.xlsx", sheet_name='Sheet1')
df4 = pd.read_excel("CALL.xlsx", sheet_name='Sheet1')

In [267]:
DELIV = df
DSERV = df2
EMP = df3
CALL = df4

In [268]:
DELIV = Columnames(DELIV, "_DELIV")
DSERV = Columnames(DSERV, "_DSERV")
EMP = Columnames(EMP, "_EMP")
CALL = Columnames(CALL, "_CALL")

In [269]:
EMP = EMP.drop(EMP.columns[[0,2,3,9,14,15,16]], axis = 1)
CALL = CALL.drop(CALL.columns[[0,1,10,11,27,28,29,31,32,33,34,35,36,37,38,44,45,46,47,49,50,52,56,57,58,59,68,69,70,72,73,74,75,76,77,78,79,80,81,83]], axis = 1)
DSERV = DSERV.drop(["Motivo_de_Pedido_DSERV","Texto_breve_de_material_DSERV","Producto_Comercial_DSERV","Frente_DSERV","Identif._externa_nota_de_entrega_DSERV","Cantidad_entrada_de_mercancías_DSERV","Nombre_Holding_DSERV","Nombre_Jerarquía_DSERV","Repr._Ventas_DSERV","Factura_DSERV","Posición_DSERV","Creado_el_DSERV","Hora_DSERV"], axis=1)

In [270]:
# Separar los dartos apartir de caracteres "-"
pedido = CALL["Pedido._CALL"].str.split('-', expand=True)
CALL["Pedido._CALL"] = pedido[0]

In [271]:
Data = DELIV.merge(DSERV.rename({'Entrega_DELIV': 'Entrega_DSERV'}, axis=1),
               left_on='Entrega_DELIV', right_on='Entrega_DSERV', how='left')

In [273]:
CALL["Pedido._CALL"] = CALL["Pedido._CALL"].astype('float64')

In [274]:
Data = Data.merge(CALL.rename({'Orden_de_Venta_DSERV': 'Pedido._CALL'}, axis=1),
               left_on='Orden_de_Venta_DSERV', right_on='Pedido._CALL', how='left')

In [275]:
Data = Data.merge(EMP.rename({'Entrega_DELIV': 'Entrega_EMP'}, axis=1),
               left_on='Entrega_DELIV', right_on='Entrega_EMP', how='left')

In [276]:
Data = Data.drop(["Pedido._CALL","Entrega_EMP"], axis=1)

In [257]:
len(DSERV['Entrega_DSERV'].unique())

10

In [258]:
len(DELIV['Entrega_DELIV'].unique())

10

In [277]:
len(Data)

10

In [259]:
DSERV = DSERV.rename(columns={'Entrega_DSERV':'Entrega_DELIV'})


In [260]:
a = DELIV.merge(DSERV, on='Entrega_DELIV', how='left')

In [245]:
a = pd.merge(DELIV,DSERV, how='left')

In [279]:
Data.columns

Index(['Centro_DELIV', 'Conductor_DELIV', 'Cantidad_entrega_DELIV',
       'Fecha_inicio_real_DELIV', 'Hora_inicio_real_DELIV',
       'Nombre_de_la_Obra_DELIV', 'Documento_de_Venta_DELIV',
       'Placa_Vehículo_1_DELIV', 'Carga_DELIV',
       'In-Plant_Hour_of_Delivery_DELIV',
       ...
       'Fecha_de_Carga_EMP', 'Hora_de_Carga_EMP', 'Hora_de_Entrada_EMP',
       'Hora_de_Fin_de_Jornada_EMP', 'Distancia_EMP', 'Tiempo_de_Viaje_EMP',
       'Volumen_EMP', 'Centro_EMP', 'Estatus_EMP', 'Turno_EMP'],
      dtype='object', length=136)

In [280]:
a.head()

,Centro_DELIV,Conductor_DELIV,Cantidad_entrega_DELIV,Fecha_inicio_real_DELIV,Hora_inicio_real_DELIV,Nombre_de_la_Obra_DELIV,Documento_de_Venta_DELIV,Placa_Vehículo_1_DELIV,Carga_DELIV,In-Plant_Hour_of_Delivery_DELIV,...,Nombre_Canal_DSERV,Nombre_Subcanal_DSERV,Nombre_Especialidad_DSERV,Offline_DSERV,Fabricación_en_Línea_DSERV,Elemento_a_Colar_DSERV,Nombre_Vicepresidencia_DSERV,Fecha_de_documento_DSERV,Pedido_cliente_DSERV,Nombre_Vendedor_DSERV
0,D105,159956,6.0,2022-11-30,15:18:42,ANTARA FASE II,8.013570e+09,CR4362,L006,2022-11-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D678,1724160,7.0,2022-11-30,16:39:41,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR4826,L001,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D105,1706888,3.0,2022-11-30,16:54:21,ANTARA FASE II,8.013570e+09,CR4343,L008,2022-11-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D678,1709505,7.0,2022-11-30,18:00:44,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR5156,L002,2022-11-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D251,1724209,7.0,2022-11-30,23:13:21,PILLM Pisos VESTA PARK,8.013569e+09,CR4458,L022,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
len(DSERV)

10

In [263]:
len(DELIV)

10

In [231]:
DELIV.columns

Index(['Centro_DELIV', 'Conductor_DELIV', 'Cantidad_entrega_DELIV',
       'Fecha_inicio_real_DELIV', 'Hora_inicio_real_DELIV',
       'Nombre_de_la_Obra_DELIV', 'Documento_de_Venta_DELIV',
       'Placa_Vehículo_1_DELIV', 'Carga_DELIV',
       'In-Plant_Hour_of_Delivery_DELIV', 'In-Plant_Hour_of_Delivery.1_DELIV',
       'Estatus_DELIV', 'Obra_DELIV', 'Entrega_DELIV', 'Nombre_1_DELIV',
       'Fecha_de_entrega_DELIV', 'In_Site_Date_DELIV', 'In_Site_Hr_DELIV',
       'Unld_Initial_Date_DELIV', 'Unld_Initial_Hr_DELIV',
       'Unld_Final_Date_DELIV', 'Unld_Final_Hr_DELIV',
       'Date_of_Assigned_Delivery_DELIV', 'Hour_of_Assigned_Delivery_DELIV',
       'Texto_breve_de_material_DELIV', 'Producto_Comercial_DELIV',
       'Orden_de_Producción_DELIV', 'Fecha_de_documento_DELIV',
       'Fecha_de_entrega.1_DELIV', 'Cliente_DELIV', 'Ticket_DELIV',
       'Fecha_de_carga_DELIV', 'Hora_de_entrega_DELIV',
       'Fecha_real_de_fin_de_la_orden_DELIV', 'Fin_real_(hora)_DELIV',
       'Tiempo_de

In [232]:
DSERV.columns

Index(['Entrega_DSERV', 'In_Site_Date_DSERV', 'In_Site_Hr_DSERV',
       'Unld_Initial_Date_DSERV', 'Unld_Initial_Hr_DSERV',
       'Unld_Final_Date_DSERV', 'Unld_Final_Hr_DSERV',
       'In-Plant_Hour_of_Delivery_DSERV', 'In-Plant_Hour_of_Delivery.1_DSERV',
       'Clase_de_documento_DSERV', 'Orden_de_Venta_DSERV',
       'Fecha_de_entrega_DSERV', 'Hora_de_entrega_DSERV',
       'Fecha_de_carga_DSERV', 'Hora_de_carga_DSERV',
       'Fecha_inicio_real_DSERV', 'Hora_inicio_real_DSERV',
       'Fecha_real_de_fin_de_la_orden_DSERV', 'Fin_real_(hora)_DSERV',
       'No._Fab._DSERV', 'Tiempo_Fabricación_DSERV', 'Fecha_factura_DSERV',
       'Nombre_1_DSERV', 'Nombre_Canal_DSERV', 'Nombre_Subcanal_DSERV',
       'Nombre_Especialidad_DSERV', 'Offline_DSERV',
       'Fabricación_en_Línea_DSERV', 'Elemento_a_Colar_DSERV',
       'Nombre_Vicepresidencia_DSERV', 'Fecha_de_documento_DSERV',
       'Pedido_cliente_DSERV', 'Nombre_Vendedor_DSERV'],
      dtype='object')

In [225]:
len(DELIV)

210010

In [226]:
len(DSERV)

303190

In [227]:
len(CALL)

3443

In [228]:
len(EMP)

3401

In [229]:
len(Data)

590580

In [211]:
Data["Pedido._CALL"]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
590575   NaN
590576   NaN
590577   NaN
590578   NaN
590579   NaN
Name: Pedido._CALL, Length: 590580, dtype: float64

In [198]:
DELIV.columns

Index(['Centro_DELIV', 'Conductor_DELIV', 'Cantidad_entrega_DELIV',
       'Fecha_inicio_real_DELIV', 'Hora_inicio_real_DELIV',
       'Nombre_de_la_Obra_DELIV', 'Documento_de_Venta_DELIV',
       'Placa_Vehículo_1_DELIV', 'Carga_DELIV',
       'In-Plant_Hour_of_Delivery_DELIV', 'In-Plant_Hour_of_Delivery.1_DELIV',
       'Estatus_DELIV', 'Obra_DELIV', 'Entrega_DELIV', 'Nombre_1_DELIV',
       'Fecha_de_entrega_DELIV', 'In_Site_Date_DELIV', 'In_Site_Hr_DELIV',
       'Unld_Initial_Date_DELIV', 'Unld_Initial_Hr_DELIV',
       'Unld_Final_Date_DELIV', 'Unld_Final_Hr_DELIV',
       'Date_of_Assigned_Delivery_DELIV', 'Hour_of_Assigned_Delivery_DELIV',
       'Texto_breve_de_material_DELIV', 'Producto_Comercial_DELIV',
       'Orden_de_Producción_DELIV', 'Fecha_de_documento_DELIV',
       'Fecha_de_entrega.1_DELIV', 'Cliente_DELIV', 'Ticket_DELIV',
       'Fecha_de_carga_DELIV', 'Hora_de_entrega_DELIV',
       'Fecha_real_de_fin_de_la_orden_DELIV', 'Fin_real_(hora)_DELIV',
       'Tiempo_de

In [14]:
SERV['Fecha_inicio_real_SERV'] = SERV['Fecha_inicio_real_SERV'].astype(str) + " " + SERV['Hora_inicio_real_SERV'].astype(str)
SERV['In-Plant_Hour_of_Delivery_SERV'] = SERV['In-Plant_Hour_of_Delivery_SERV'].astype(str) + " " + SERV['In-Plant_Hour_of_Delivery.1_SERV'].astype(str)
SERV['In_Site_Date_SERV'] = SERV['In_Site_Date_SERV'].astype(str) + " " + SERV['In_Site_Hr_SERV'].astype(str)
SERV['Unld_Initial_Date_SERV'] = SERV['Unld_Initial_Date_SERV'].astype(str) + " " + SERV['Unld_Initial_Hr_SERV'].astype(str)
SERV['Unld_Final_Date_SERV'] = SERV['Unld_Final_Date_SERV'].astype(str) + " " + SERV['Unld_Final_Hr_SERV'].astype(str)
SERV['Date_of_Assigned_Delivery_SERV'] = SERV['Date_of_Assigned_Delivery_SERV'].astype(str) + " " + SERV['Hour_of_Assigned_Delivery_SERV'].astype(str)
SERV['Fecha_de_entrega.1_SERV'] = SERV['Fecha_de_entrega.1_SERV'].astype(str) + " " + SERV['Hora_de_entrega_SERV'].astype(str)
SERV['Fecha_real_de_fin_de_la_orden_SERV'] = SERV['Fecha_real_de_fin_de_la_orden_SERV'].astype(str) + " " + SERV['Fin_real_(hora)_SERV'].astype(str)
SERV['Fecha_TJST_SERV'] = SERV['Fecha_TJST_SERV'].astype(str) + " " + SERV['Hora_TJST_SERV'].astype(str)
SERV['Fecha_TOPL_SERV'] = SERV['Fecha_TOPL_SERV'].astype(str) + " " + SERV['Hora_TOPL_SERV'].astype(str)
SERV['Fecha_de_carga_SERV'] = SERV['Fecha_de_carga_SERV'].astype(str) + " " + SERV['Hora_de_carga_SERV'].astype(str)

In [15]:
SERV = SERV.drop(['Hora_inicio_real_SERV','In-Plant_Hour_of_Delivery.1_SERV','In_Site_Hr_SERV','Unld_Initial_Hr_SERV','Unld_Final_Hr_SERV','Hour_of_Assigned_Delivery_SERV','Hora_de_entrega_SERV','Fin_real_(hora)_SERV','Hora_TJST_SERV','Hora_TOPL_SERV','Hora_de_carga_SERV'], axis=1)